# Data Sample

In [1]:
import pandas as pd 
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import shapely.geometry
import numpy as np
import json


pd.set_option('display.max_columns', None)

In [9]:
res = pd.read_pickle("data/computed/matches_clean_EWT_delay.pkl")
data = res
df = res

In [4]:
res = pd.read_pickle("data/computed/matches_clean_EWT_delay.pkl")

In [4]:
df.columns

Index(['trip_id', 'theoretical_time', 'stop_id', 'stop_sequence', 'stop_name',
       'route_id', 'service_id', 'trip_headsign', 'direction_id',
       'route_short_name', 'route_long_name', 'th_time_sec', 'headway_th',
       'clusters', 'cluster_agg_value', 'regularity', 'start_date_ft',
       'end_date_ft', 'date_normalized', 'dist_m', 'distance_from_point',
       'stop_id_cleaned', 'stop_id__terminus', 'stop_name__terminus',
       'stop_id_cleaned__terminus', 'date_label', 'ligne_cleaned', 'mode',
       'real_time_sec', 'adj_real_time', 'vehicule_id', 'approx_rec',
       'headway_real', 'SWT', 'AWT', 'EWT', 'EWT_label', 'delay',
       'delay_label_strict', 'delay_label_60', 'delay_label_120', 'hour',
       'day'],
      dtype='object')

In [75]:
#data = pd.read_pickle("data/computed/matches_EWT_delay_subsampled.pkl")
#stops_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")
#lines_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops = pd.read_csv("data/timetables/gtfs3Sept/stops.txt")


In [4]:
coord = stops[["stop_id", "stop_lat", "stop_lon"]]

In [ ]:
data.route_long_name.unique()

In [ ]:
data.to_csv("data.csv")

In [ ]:
data_sample = data[(data.route_short_name == "1") & (data.stop_name == 'DE BROUCKERE') & (data.direction_id == 1) & (data.date == '06-09-2021')]

In [ ]:
data_sample = data_sample.sort_values(by="theoretical_time")

In [ ]:
data_sample

In [ ]:
data_sample[data_sample.delay == 561]

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(data_sample,
        x="real_time_sec",
        y="delay",
        color="delay_label",
    )

In [ ]:
fig

In [94]:
coord = stops[["stop_id", "stop_lat", "stop_lon"]]

In [11]:
test = res.groupby(["stop_id"]).agg({"delay" : ["median"]}).reset_index()
test.columns = test.columns.get_level_values(0)
test = test.merge(
    coord,
    "left",
    "stop_id"
    
)
test

,stop_id,delay,stop_lat,stop_lon
0,0089,37.0,50.838006,4.408970
1,0470F,25.0,50.863666,4.329612
2,0471,36.0,50.863732,4.329236
3,0472,41.0,50.863543,4.329023
4,0473F,49.5,50.863418,4.330031
...,...,...,...,...
2270,9972F,185.0,50.796526,4.348173
2271,9979B,7.0,50.838552,4.336295
2272,9983,132.0,50.844760,4.417568
2273,9986F,6.0,50.812075,4.353740


In [ ]:
test[test.delay<0]

Type de map 1

In [36]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(test, lat='stop_lat', lon='stop_lon', z='delay', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=9,
                        mapbox_style="white-bg")

fig.update_coloraxes(showscale=False)

fig.show()

Type de map 2

In [32]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.scatter_mapbox(test, lat="stop_lat", lon="stop_lon", hover_name="stop_id",
                        color="delay", zoom=10)
fig.update_layout(mapbox_style="white-bg")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(showlegend=False)
fig.show()

Type de map 3

In [ ]:
stops_sh

La méthode en bas marche seulement si c'est des polygones !  
--> px.choropleth_mapbox accepts the geometry of a GeoPandas data frame as the input to geojson if the geometry contains polygons.

In [ ]:
fig = px.choropleth_mapbox(stops_sh,
                           geojson=stops_sh.geometry,
                           locations=stops_sh.stop_id,
                           center={"lat": 50.85, "lon": 4.45},
                           mapbox_style="open-street-map",
                           zoom=10)
fig.show()

Tentative dash leaflet - un peu ratée mais je crois comprendre pourquoi. Quand on transforme en geojson, les coordonées n'ont pas un format très standard

In [ ]:
layer = dl.GeoJSON(json.loads(lines_sh.to_json()))
app = JupyterDash()
app.layout = dl.Map(
    [layer, dl.TileLayer()]
)
#app.run_server(mode="inline")

In [ ]:
gs = json.loads(lines_sh.to_json())
fig = go.Figure(
    data=[
        go.Scattermapbox(
            lat=np.array(feature["geometry"]["coordinates"])[:, 1],
            lon=np.array(feature["geometry"]["coordinates"])[:, 0],
            mode="lines",
            line=dict(width=8, color="#F00")
        )
        for feature in gs["features"]
    ]
)

fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    mapbox=go.layout.Mapbox(
        style="stamen-terrain", 
        zoom=10, 
        center_lat = 40.5,
        center_lon = -105.08,
    )
)
fig.show()

In [ ]:
gs

## Information par ligne par jour de la semaine

Ici on peut aller par ligne, par type de jour de la semaine, par cluster etc

In [96]:
res

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN,8.654762,11.624663,2.969901,irregular,0,nan,nan,nan,5,7
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667,8.654762,11.624663,2.969901,irregular,796,lat,lat,lat,5,7
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000,8.654762,11.624663,2.969901,irregular,-29,adv,nan,nan,5,7
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000,8.654762,11.624663,2.969901,irregular,4,nan,nan,nan,6,7
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333,8.654762,11.624663,2.969901,irregular,18,nan,nan,nan,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333,6.382353,7.235564,0.853211,regular,-82,adv,adv,nan,8,17
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000,6.382353,7.235564,0.853211,regular,-1,adv,nan,nan,8,17
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667,6.382353,7.235564,0.853211,regular,525,lat,lat,lat,8,17
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333,6.382353,7.235564,0.853211,regular,86,lat,lat,nan,8,17


In [105]:
sample_7 = res[(res.route_short_name == "7") & (res.date_label == "workdays") & (res.direction_id == 0)]

In [106]:
sample_7

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
87019,113338976237534000,04:42:00,0806,8,MONTGOMERY,6,237534000,HEYSEL,0,7,VANDERKINDERE - HEYSEL,16920,NaN,cluster_0,6.0,1,2021-09-01,2021-09-17,07-09-2021,452.382581,52.0,806,9051F,HEYSEL,9051,workdays,7,T,16897,04:41:37,179201,NaN,NaN,3.779762,4.451198,0.671436,regular,-23,adv,nan,nan,4,7
87020,113338836237534000,04:54:00,0806,8,MONTGOMERY,6,237534000,HEYSEL,0,7,VANDERKINDERE - HEYSEL,17640,12.0,cluster_0,6.0,1,2021-09-01,2021-09-17,07-09-2021,452.382581,0.0,806,9051F,HEYSEL,9051,workdays,7,T,17542,04:52:22,177801,NaN,10.750000,3.779762,4.451198,0.671436,regular,-98,adv,adv,nan,4,7
87021,113338837237534000,05:04:00,0806,8,MONTGOMERY,6,237534000,HEYSEL,0,7,VANDERKINDERE - HEYSEL,18240,10.0,cluster_0,6.0,1,2021-09-01,2021-09-17,07-09-2021,452.382581,292.0,806,9051F,HEYSEL,9051,workdays,7,T,18257,05:04:17,177811,NaN,11.916667,3.779762,4.451198,0.671436,regular,17,nan,nan,nan,5,7
87022,113338840237534000,05:14:00,0806,8,MONTGOMERY,6,237534000,HEYSEL,0,7,VANDERKINDERE - HEYSEL,18840,10.0,cluster_0,6.0,1,2021-09-01,2021-09-17,07-09-2021,452.382581,44.0,806,9051F,HEYSEL,9051,workdays,7,T,18780,05:13:00,177841,NaN,8.716667,3.779762,4.451198,0.671436,regular,-60,adv,nan,nan,5,7
87023,113338834237534000,05:23:00,0806,8,MONTGOMERY,6,237534000,HEYSEL,0,7,VANDERKINDERE - HEYSEL,19380,9.0,cluster_0,6.0,1,2021-09-01,2021-09-17,07-09-2021,452.382581,0.0,806,9051F,HEYSEL,9051,workdays,7,T,19466,05:24:26,177781,NaN,11.433333,3.779762,4.451198,0.671436,regular,86,lat,lat,nan,5,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433703,113460673238160601,24:26:00,9963F,1,CHURCHILL,6,238160601,BUYL,0,7,VANDERKINDERE - HEYSEL,87960,10.0,cluster_1,8.0,1,2021-09-12,2021-09-12,12-09-2021,NaN,0.0,9963,2351,BUYL,2351,workdays,7,T,88418,00:33:38,182130,NaN,39.383333,4.210425,8.406999,4.196574,irregular,458,lat,lat,lat,24,12
3433704,113460667238160601,24:30:00,9963F,3,CHURCHILL,6,238160601,BUYL,0,7,VANDERKINDERE - HEYSEL,88200,4.0,cluster_1,8.0,1,2021-09-12,2021-09-12,12-09-2021,82.969874,0.0,9963,2351,BUYL,2351,workdays,7,T,88418,00:33:38,182124,NaN,0.000000,4.210425,8.406999,4.196574,irregular,218,lat,lat,lat,24,12
3433705,113460663238160601,24:41:00,9963F,1,CHURCHILL,6,238160601,BUYL,0,7,VANDERKINDERE - HEYSEL,88860,11.0,cluster_1,8.0,1,2021-09-12,2021-09-12,12-09-2021,NaN,0.0,9963,2351,BUYL,2351,workdays,7,T,89223,00:47:03,182120,NaN,0.000000,4.210425,8.406999,4.196574,irregular,363,lat,lat,lat,24,12
3433706,113460698238160601,24:46:00,9963F,3,CHURCHILL,6,238160601,BUYL,0,7,VANDERKINDERE - HEYSEL,89160,5.0,cluster_1,8.0,1,2021-09-12,2021-09-12,12-09-2021,82.969874,0.0,9963,2351,BUYL,2351,workdays,7,T,89223,00:47:03,182155,NaN,13.416667,4.210425,8.406999,4.196574,irregular,63,lat,lat,nan,24,12


En somme, pour avoir l'impact total causé par la ligne

In [99]:
s_7_s = sample_7[["stop_id", "delay"]].groupby("stop_id", as_index = False).sum("delay").merge(
    coord,
    "left",
    "stop_id"
).merge(sample_7[["stop_id", "stop_name"]], "left", "stop_id")

In [17]:
s_7_s

,stop_id,delay,stop_lat,stop_lon,stop_name
0,0801,169996,50.837123,4.407182,MONTGOMERY
1,0801,169996,50.837123,4.407182,MONTGOMERY
2,0801,169996,50.837123,4.407182,MONTGOMERY
3,0801,169996,50.837123,4.407182,MONTGOMERY
4,0801,169996,50.837123,4.407182,MONTGOMERY
...,...,...,...,...,...
25838,9969F,195719,50.811985,4.353655,CHURCHILL
25839,9969F,195719,50.811985,4.353655,CHURCHILL
25840,9969F,195719,50.811985,4.353655,CHURCHILL
25841,9969F,195719,50.811985,4.353655,CHURCHILL


In [18]:
px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(s_7_s, lat='stop_lat', lon='stop_lon', z='delay', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=10,
                        color_continuous_scale = "portland" , 
                        mapbox_style="light",
                        hover_name="stop_name", hover_data=["delay"])
fig.show()

En médiane, pour avoir l'impact médian par arrêt

In [107]:
s_7_m = sample_7[["stop_id", "EWT"]].groupby("stop_id", as_index = False).median("EWT").merge(
    coord,
    "left",
    "stop_id"
)

In [108]:

px.set_mapbox_access_token("pk.eyJ1IjoibWpkYW91ZGkiLCJhIjoiY2xibm54OThyMGdyOTNvcnhqeTYyZmRuYiJ9.rfxe3z8triwA5yvV1XZA-A")
fig = px.density_mapbox(s_7_m, lat='stop_lat', lon='stop_lon', z='EWT', radius=10,
                        center=dict(lat=50.85, lon=4.45), zoom=10,
                        color_continuous_scale = "portland" , 
                        mapbox_style="stamen-terrain")
fig.show()

Observation tram 7: 
   <br><br/> - beaucoup de retard aux alentours de vdk, soit ils impactent la suite (direction heysel); soit les retards sont vers la fin de la ligne (direction vdk)

In [ ]:
res

In [5]:
# Number of stops
round(res.stop_id.unique().shape[0])

2275

In [6]:
# Number of lines
round(res.route_short_name.unique().shape[0])

74

In [11]:
# On time positionning (idealemnent, rajouter un threshold with thhresh hold)
round((res[res.regularity == 0].delay_label_120.value_counts()/res[res.regularity == 0].shape[0])*100, 2).to_dict()

{'nan': 54.37, 'lat': 39.83, 'adv': 5.8}

In [79]:
_ = res[(res.regularity == 0)][["stop_name", "delay"]].groupby("stop_name", as_index = False).count().sort_values("delay", ascending = True).merge(
    res[(res.regularity == 0) & (res.delay_label == "lat")][["stop_name", "delay"]].groupby("stop_name", as_index = False).count().sort_values("delay", ascending = True),
    "left",
    "stop_name"
)
_["Probs"] = round((_.delay_y/_.delay_x)*100)
_.sort_values("Probs", ascending=False).iloc[0].Probs

91.0

In [28]:
res[
    (res.regularity == 0)
    & (res.delay_label_strict == "lat")
][["stop_name", "route_short_name", "delay"]].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=False).merge(
    res[(res.regularity == 0)][
    ["stop_name", "delay", "route_short_name"]
].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=True),
"left",
["stop_name", "route_short_name"]
)

,stop_name,route_short_name,delay_x,delay_y
0,DIAMANT,21,2355,3567
1,MICHEL-ANGE,21,1972,2672
2,FROISSART,21,1898,2644
3,SCHUMAN,21,1855,2672
4,GUEUX,21,1828,2634
...,...,...,...,...
1771,MONTGOMERY,25,1,17
1772,DE HEYN,83,1,1
1773,BOONDAEL GARE,25,1,31
1774,MARIE-JOSE,25,1,37


In [30]:
def get_probs():
    _ = res[
            (res.regularity == 0)
            & (res.delay_label_strict == "lat")
        ][["stop_name", "route_short_name", "delay"]].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=False).merge(
            res[(res.regularity == 0)][
            ["stop_name", "delay", "route_short_name"]
        ].groupby(["stop_name", "route_short_name"], as_index=False).count().sort_values("delay", ascending=True),
        "left",
        ["stop_name", "route_short_name"]
        )
    _["Probs"] = round((_.delay_x / _.delay_y) * 100)
    return _[_.delay_y>100].sort_values(by="Probs", ascending= False)

get_probs()



,stop_name,route_short_name,delay_x,delay_y,Probs
464,HERNALSTEEN,76,488,538,91.0
488,MARMOTTE,76,475,527,90.0
485,VIOLETTES,76,475,533,89.0
1261,BRABANCONNE,64,179,204,88.0
509,PLAINE DES SPORTS,76,461,526,88.0
...,...,...,...,...,...
1599,LEGRAND,93,38,288,13.0
1540,GARE DU MIDI,78,55,448,12.0
1715,ESPLANADE,7,13,118,11.0
1626,COTEAUX,25,32,307,10.0


In [9]:
# Number of vehicule
veh = res.trip_id.unique().shape[0]
"number of vehicule", veh


('number of vehicule', 40387)

In [ ]:
fig = px.histogram(res, x="delay_label", color="mode")
fig.show()

In [ ]:
mode = "B"
fig = px.histogram(res[(res.delay.quantile(0.9) > res.delay) & (res.delay > res.delay.quantile(0.05)) & (res["mode"] == mode)], x="delay", histnorm='percent')
fig.show()

In [83]:
res["EWT_label"] = np.where(res["EWT"] >= 1, "irregular", "regular")


In [85]:
res.to_pickle("data/computed/matches_EWT_delay_reglab.pkl")

In [90]:
round((res[res.regularity == 1].EWT_label.value_counts()/res[res.regularity == 1].shape[0])*100, 2).to_dict()

{'irregular': 64.76, 'regular': 35.24}

In [91]:
res[res.regularity == 1].groupby(by = ["route_short_name"], as_index = False).apply(lambda x: x["EWT"].median())

,route_short_name,None
0,1,0.670729
1,12,1.200639
2,13,1.249259
3,14,1.662095
4,17,1.618177
...,...,...
68,92,1.614077
69,93,1.808726
70,95,1.682855
71,97,2.093906


In [43]:
df = res

data = res

In [15]:
df[(df.stop_name == "DIEWEG") & (df.route_short_name == "97") & (df.date_normalized == "10-09-2021")& (df.direction_id == 1)].sort_values("headway_real", ascending=False).to_csv("test.csv")

In [49]:
df[(df.route_short_name == "97") & (df.regularity == 1)].sort_values("EWT", ascending=False)

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120
1986731,113078764236711001,15:04:49,3002,13,DGHR DOMAINE MIL.,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,54289,8.0,cluster_2,8.0,1,2021-09-01,2021-09-17,09-09-2021,343.758636,470.0,3002,2299,HEEMBEEK,2299,workdays,47,B,54650,15:10:50,95021,NaN,477.250000,4.00,238.625000,234.625000,irregular,361,lat,lat,lat
1266436,113078853236711001,19:22:00,2352,11,HOP. MILITAIRE,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,69720,9.0,cluster_4,9.5,1,2021-09-01,2021-09-17,06-09-2021,329.018556,0.0,2352,2299,HEEMBEEK,2299,workdays,47,B,69585,19:19:45,95908,1.0,7.100000,4.18,167.486156,163.306156,irregular,-135,adv,adv,adv
1266434,113078782236711001,19:05:00,2352,11,HOP. MILITAIRE,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,68700,8.0,cluster_4,9.5,1,2021-09-01,2021-09-17,06-09-2021,329.018556,0.0,2352,2299,HEEMBEEK,2299,workdays,47,B,69159,19:12:39,95198,NaN,341.783333,4.18,167.486156,163.306156,irregular,459,lat,lat,lat
1266435,113078858236711001,19:13:00,2352,2,HOP. MILITAIRE,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,69180,8.0,cluster_4,9.5,1,2021-09-01,2021-09-17,06-09-2021,105.900000,0.0,2352,2299,HEEMBEEK,2299,workdays,47,B,69159,19:12:39,95958,NaN,0.000000,4.18,167.486156,163.306156,irregular,-21,adv,nan,nan
1266471,113078780236711001,18:19:00,2352,11,HOP. MILITAIRE,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,65940,6.0,cluster_3,6.0,1,2021-09-01,2021-09-17,13-09-2021,329.018556,52.0,2352,2299,HEEMBEEK,2299,workdays,47,B,65927,18:18:47,95183,1.0,3.466667,3.00,162.911124,159.911124,irregular,-13,adv,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986804,113078739236711001,08:44:49,3002,13,DGHR DOMAINE MIL.,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,31489,6.0,cluster_1,6.0,1,2021-09-01,2021-09-17,14-09-2021,343.758636,460.0,3002,2299,HEEMBEEK,2299,workdays,47,B,31467,08:44:27,94774,NaN,0.000000,3.00,NaN,NaN,regular,-22,nan,nan,nan
1986825,113078748236711001,11:09:13,3002,4,DGHR DOMAINE MIL.,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,40153,8.0,cluster_2,8.0,1,2021-09-01,2021-09-17,15-09-2021,343.758636,442.0,3002,2299,HEEMBEEK,2299,workdays,47,B,40502,11:15:02,94865,NaN,NaN,4.00,NaN,NaN,regular,349,lat,lat,lat
1986826,113078800236711001,11:17:13,3002,13,DGHR DOMAINE MIL.,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,40633,8.0,cluster_2,8.0,1,2021-09-01,2021-09-17,15-09-2021,343.758636,442.0,3002,2299,HEEMBEEK,2299,workdays,47,B,40502,11:15:02,95385,NaN,0.000000,4.00,NaN,NaN,regular,-131,adv,adv,adv
1986827,113078738236711001,08:32:49,3002,13,DGHR DOMAINE MIL.,47,236711001,HEEMBEEK,1,47,HEEMBEEK - VILVOORDE STATION,30769,6.0,cluster_1,6.0,1,2021-09-01,2021-09-17,16-09-2021,343.758636,459.0,3002,2299,HEEMBEEK,2299,workdays,47,B,31083,08:38:03,94766,NaN,NaN,3.00,NaN,NaN,regular,314,lat,lat,lat


In [3]:
df[df.regularity == 1][["stop_name","route_short_name", "EWT"]].groupby(by = ["stop_name", "route_short_name"], as_index = False).median().sort_values("EWT", ascending = False)

,stop_name,route_short_name,EWT
531,FORESTERIE,17,4.504705
1494,VEEWEYDE,46,4.287544
987,MOENSBERG GARE,43,4.184163
1161,PORTE DE NAMUR,64,4.165267
562,GARE CENTRALE,89,4.047642
...,...,...,...
270,CHAUMONTEL,69,0.062204
1130,PICARDIE,69,-0.088166
1592,ZENOBE GRAMME,69,-0.487199
1477,VANDENHOVEN,69,-0.568637


In [ ]:
EWT_line = df[df.regularity == 1].groupby(by = ["route_short_name"]).apply(lambda x: x["EWT"].median())
EWT_line.name = 'EWT_line'
df = df.merge(
    EWT_line,
    'left',
    'route_short_name'
)
df["EWT_line_label"] = np.where(df["EWT_line"] >= 1, "irregular", "regular")
line_prob= round((df[df.regularity == 1].EWT_line_label.value_counts()/df[df.regularity == 1].shape[0])*100, 2).to_dict()
worst_line = df[(df.regularity == 1) & (df.EWT_line_label == "irregular")].sort_values(by= 'EWT_line', ascending = False).iloc[0].route_short_name

In [45]:
data[data.regularity == 1][["route_short_name", "EWT"]].groupby(by=["route_short_name"], as_index=False).median().sort_values("EWT", ascending=False).iloc[0]

route_short_name          41
EWT                 3.275533
Name: 20, dtype: object

In [4]:
data[data.regularity == 0]

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN,8.654762,11.624663,2.969901,irregular,0,nan,nan,nan
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667,8.654762,11.624663,2.969901,irregular,796,lat,lat,lat
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000,8.654762,11.624663,2.969901,irregular,-29,adv,nan,nan
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000,8.654762,11.624663,2.969901,irregular,4,nan,nan,nan
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333,8.654762,11.624663,2.969901,irregular,18,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333,6.382353,7.235564,0.853211,regular,-82,adv,adv,nan
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000,6.382353,7.235564,0.853211,regular,-1,adv,nan,nan
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667,6.382353,7.235564,0.853211,regular,525,lat,lat,lat
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333,6.382353,7.235564,0.853211,regular,86,lat,lat,nan


In [18]:
data["hour"] = data[data.regularity == 0].theoretical_time.apply(lambda x:int(x.split(":")[0]))
data["day"] = data[data.regularity == 0].date_normalized.apply(lambda x:int(x.split("-")[0]))

In [11]:
data["hour"]

0          5.0
1          5.0
2          5.0
3          6.0
4          6.0
          ... 
3434406    8.0
3434407    8.0
3434408    8.0
3434409    8.0
3434410    8.0
Name: hour, Length: 3434411, dtype: float64

In [19]:
t = data[["hour", "mode", "delay"]].groupby(["hour", "mode"], as_index=False).median()
t = t[t.hour <= 24]

fig = px.line(t, x="hour", y="delay", color='mode').update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
fig.show()

In [23]:
t = data[["day", "mode", "delay"]].groupby(["day", "mode"], as_index=False).median()

fig = px.line(t, x="day", y="delay", color='mode').update_layout({
    'plot_bgcolor': 'rgba(0,0,0,0)',
    'paper_bgcolor': 'rgba(0,0,0,0)'
})
fig.show()

In [5]:
df

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN,8.654762,11.624663,2.969901,irregular,0,nan,nan,nan,5,7
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667,8.654762,11.624663,2.969901,irregular,796,lat,lat,lat,5,7
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000,8.654762,11.624663,2.969901,irregular,-29,adv,nan,nan,5,7
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000,8.654762,11.624663,2.969901,irregular,4,nan,nan,nan,6,7
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333,8.654762,11.624663,2.969901,irregular,18,nan,nan,nan,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333,6.382353,7.235564,0.853211,regular,-82,adv,adv,nan,8,17
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000,6.382353,7.235564,0.853211,regular,-1,adv,nan,nan,8,17
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667,6.382353,7.235564,0.853211,regular,525,lat,lat,lat,8,17
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333,6.382353,7.235564,0.853211,regular,86,lat,lat,nan,8,17


In [72]:
data

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
0,112960457236299000,05:17:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,19020,NaN,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,60.0,89,9556,TERVUREN STATION,9556,workdays,44,T,19020,05:17:00,76753,NaN,NaN,8.654762,11.624663,2.969901,irregular,0,nan,nan,nan,5,7
1,112961236236299000,05:39:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,20340,22.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21136,05:52:16,79193,NaN,35.266667,8.654762,11.624663,2.969901,irregular,796,lat,lat,lat,5,7
2,112960463236299000,05:56:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,21360,17.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,101.0,89,9556,TERVUREN STATION,9556,workdays,44,T,21331,05:55:31,76783,NaN,3.250000,8.654762,11.624663,2.969901,irregular,-29,adv,nan,nan,5,7
3,112960468236299000,06:12:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,22320,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,22324,06:12:04,76813,NaN,16.550000,8.654762,11.624663,2.969901,irregular,4,nan,nan,nan,6,7
4,112960472236299000,06:28:00,0089,1,MONTGOMERY,40,236299000,TERVUREN STATION,0,44,MONTGOMERY - TERVUREN STATION,23280,16.0,cluster_0,16.0,0,2021-09-01,2021-09-17,07-09-2021,NaN,0.0,89,9556,TERVUREN STATION,9556,workdays,44,T,23298,06:28:18,76833,NaN,16.233333,8.654762,11.624663,2.969901,irregular,18,nan,nan,nan,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3434406,113242700237156051,08:01:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,28860,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,406.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,28778,07:59:38,91004,NaN,13.383333,6.382353,7.235564,0.853211,regular,-82,adv,adv,nan,8,17
3434407,113242701237156051,08:13:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,29580,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,126.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,29579,08:12:59,91014,NaN,13.350000,6.382353,7.235564,0.853211,regular,-1,adv,nan,nan,8,17
3434408,113242702237156051,08:26:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,30360,13.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,0.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,30885,08:34:45,91024,NaN,21.766667,6.382353,7.235564,0.853211,regular,525,lat,lat,lat,8,17
3434409,113242703237156051,08:38:00,9979B,1,BARA,30,237156051,MOORTEBEEK,1,46,MOORTEBEEK - GLIBERT,31080,12.0,cluster_0,13.0,0,2021-09-03,2021-09-17,17-09-2021,NaN,562.0,9979,1201,MOORTEBEEK,1201,workdays,46,B,31166,08:39:26,91034,NaN,4.683333,6.382353,7.235564,0.853211,regular,86,lat,lat,nan,8,17


In [44]:
sample = data[
    (data.route_short_name == "1")
        & (data.stop_name == "DE BROUCKERE")
        & (data.trip_headsign == "GARE DE L'OUEST")
        & (data.date_normalized == "11-09-2021")
        & (data.service_id == 236269500)
        & (data.regularity == 1)
]
sample

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
3072982,112947633236269500,08:21:52,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30112,12.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30088,08:21:28,216,NaN,11.366667,5.167114,6.021220,0.854106,regular,-24,nan,nan,nan,8,11
3072983,112947642236269500,08:33:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30831,11.983333,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30932,08:35:32,234,NaN,14.066667,5.167114,6.021220,0.854106,regular,101,lat,lat,nan,8,11
3072984,112947649236269500,08:43:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,31431,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,31434,08:43:54,248,NaN,8.366667,5.167114,6.021220,0.854106,regular,3,nan,nan,nan,8,11
3072985,112947647236269500,08:53:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32031,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32955,09:09:15,244,1.0,5.566667,5.167114,6.021220,0.854106,regular,924,lat,lat,lat,8,11
3072986,112947281236269500,09:03:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32631,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32621,09:03:41,154,NaN,19.783333,5.167114,6.021220,0.854106,regular,-10,nan,nan,nan,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073083,112947676236269500,22:53:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,82433,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,82433,22:53:53,302,NaN,10.500000,5.471447,7.983907,2.512461,irregular,0,nan,nan,nan,22,11
3073084,112949538236269500,23:03:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,83033,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,83024,23:03:44,498,NaN,9.850000,5.471447,7.983907,2.512461,irregular,-9,nan,nan,nan,23,11
3073086,112947678236269500,23:33:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,84833,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,84793,23:33:13,306,NaN,8.933333,5.471447,7.983907,2.512461,irregular,-40,nan,nan,nan,23,11
3073087,112947668236269500,23:48:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,85733,15.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,85725,23:48:45,286,NaN,15.533333,5.471447,7.983907,2.512461,irregular,-8,nan,nan,nan,23,11


In [45]:
sample["EWT"]=sample.EWT.round(2)

C:\Users\HAI60CW\AppData\Local\Temp\ipykernel_7548\374635364.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
sample

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
3072982,112947633236269500,08:21:52,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30112,12.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30088,08:21:28,216,NaN,11.366667,5.167114,6.021220,0.85,regular,-24,nan,nan,nan,8,11
3072983,112947642236269500,08:33:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30831,11.983333,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30932,08:35:32,234,NaN,14.066667,5.167114,6.021220,0.85,regular,101,lat,lat,nan,8,11
3072984,112947649236269500,08:43:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,31431,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,31434,08:43:54,248,NaN,8.366667,5.167114,6.021220,0.85,regular,3,nan,nan,nan,8,11
3072985,112947647236269500,08:53:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32031,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32955,09:09:15,244,1.0,5.566667,5.167114,6.021220,0.85,regular,924,lat,lat,lat,8,11
3072986,112947281236269500,09:03:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32631,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32621,09:03:41,154,NaN,19.783333,5.167114,6.021220,0.85,regular,-10,nan,nan,nan,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073083,112947676236269500,22:53:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,82433,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,82433,22:53:53,302,NaN,10.500000,5.471447,7.983907,2.51,irregular,0,nan,nan,nan,22,11
3073084,112949538236269500,23:03:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,83033,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,83024,23:03:44,498,NaN,9.850000,5.471447,7.983907,2.51,irregular,-9,nan,nan,nan,23,11
3073086,112947678236269500,23:33:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,84833,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,84793,23:33:13,306,NaN,8.933333,5.471447,7.983907,2.51,irregular,-40,nan,nan,nan,23,11
3073087,112947668236269500,23:48:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,85733,15.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,85725,23:48:45,286,NaN,15.533333,5.471447,7.983907,2.51,irregular,-8,nan,nan,nan,23,11


In [15]:
sample[["headway_th", "headway_real", "regularity"]]

,headway_th,headway_real,regularity
3072982,12.000000,11.366667,1
3072983,11.983333,14.066667,1
3072984,10.000000,8.366667,1
3072985,10.000000,5.566667,1
3072986,10.000000,19.783333,1
...,...,...,...
3073083,10.000000,10.500000,1
3073084,10.000000,9.850000,1
3073086,10.000000,8.933333,1
3073087,15.000000,15.533333,1


In [48]:
fig = px.scatter(
    sample,
    x="theoretical_time",
    y="headway_th",
    color="clusters",
    hover_data=["EWT"],
    labels=dict(headway_th='Theoretical Headway (min)', theoretical_time = 'Scheduled Time'),
)
fig.update_yaxes(
    range=(0, max(sample.headway_real)+5),
    constrain = "domain"
)

In [120]:
max(sample.headway_real)

43.06666666666667

In [49]:
fig = px.scatter(
    sample,
    x="theoretical_time",
    y="headway_real",
    color="clusters",
    hover_data={'EWT':':.2f'},
    labels=dict(headway_real='Observed Headway (min)', adj_real_time = 'Observed Time')
)
fig.update_yaxes(
    range=(0, max(sample.headway_real)+5),
    constrain = "domain"
)

In [47]:
sample

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
3072982,112947633236269500,08:21:52,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30112,12.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30088,08:21:28,216,NaN,11.366667,5.167114,6.021220,0.85,regular,-24,nan,nan,nan,8,11
3072983,112947642236269500,08:33:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,30831,11.983333,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,30932,08:35:32,234,NaN,14.066667,5.167114,6.021220,0.85,regular,101,lat,lat,nan,8,11
3072984,112947649236269500,08:43:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,31431,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,31434,08:43:54,248,NaN,8.366667,5.167114,6.021220,0.85,regular,3,nan,nan,nan,8,11
3072985,112947647236269500,08:53:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32031,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32955,09:09:15,244,1.0,5.566667,5.167114,6.021220,0.85,regular,924,lat,lat,lat,8,11
3072986,112947281236269500,09:03:51,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,32631,10.000000,cluster_1,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,32621,09:03:41,154,NaN,19.783333,5.167114,6.021220,0.85,regular,-10,nan,nan,nan,9,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3073083,112947676236269500,22:53:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,82433,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,82433,22:53:53,302,NaN,10.500000,5.471447,7.983907,2.51,irregular,0,nan,nan,nan,22,11
3073084,112949538236269500,23:03:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,83033,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,83024,23:03:44,498,NaN,9.850000,5.471447,7.983907,2.51,irregular,-9,nan,nan,nan,23,11
3073086,112947678236269500,23:33:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,84833,10.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,84793,23:33:13,306,NaN,8.933333,5.471447,7.983907,2.51,irregular,-40,nan,nan,nan,23,11
3073087,112947668236269500,23:48:53,8011,16,DE BROUCKERE,2,236269500,GARE DE L'OUEST,1,1,GARE DE L'OUEST - STOCKEL,85733,15.000000,cluster_3,10.0,1,2021-09-04,2021-09-18,11-09-2021,588.817544,0.0,8011,8731,GARE DE L'OUEST,8731,saturday,1,M,85725,23:48:45,286,NaN,15.533333,5.471447,7.983907,2.51,irregular,-8,nan,nan,nan,23,11


In [70]:
card = ""
for i in sample.clusters.unique():
    j = round(sample[sample.clusters == i].EWT.iloc[0], 2)
    card += "EWT for " + str(i) + " : " + str(j) + " min\n"
x = print(card)

EWT for cluster_1 : 0.85 min
EWT for cluster_2 : 1.15 min
EWT for cluster_3 : 2.51 min



# Maps overview segment analysis

In [77]:
#data = pd.read_pickle("data/computed/matches_EWT_delay_subsampled.pkl")
#stops_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_STOPS.shp")
#lines_sh = gpd.read_file("data/map/2109_STIB_MIVB_Network/ACTU_LINES.shp")
stops = pd.read_csv("data/timetables/gtfs3Sept/stops.txt")


In [78]:
coord = stops[["stop_id", "stop_lat", "stop_lon"]]

In [82]:
total = data[["stop_id", "delay"]].groupby("stop_id", as_index = False).sum("delay").merge(
    coord,
    "left",
    "stop_id"
).merge(data[["stop_id", "stop_name"]].drop_duplicates(), "left", "stop_id")

In [83]:
total

,stop_id,delay,stop_lat,stop_lon,stop_name
0,0089,49041,50.838006,4.408970,MONTGOMERY
1,0470F,22191,50.863666,4.329612,SIMONIS
2,0471,282787,50.863732,4.329236,SIMONIS
3,0472,195762,50.863543,4.329023,SIMONIS
4,0501,144880,50.851494,4.352876,DE BROUCKERE
...,...,...,...,...,...
2007,9959F,226,50.811688,4.354123,CHURCHILL
2008,9963F,5438,50.811841,4.354222,CHURCHILL
2009,9969F,19074,50.811985,4.353655,CHURCHILL
2010,9972F,38590,50.796526,4.348173,DIEWEG


In [88]:
fig = px.density_mapbox(
        total,
        lat="stop_lat",
        lon="stop_lon",
        z="delay",
        radius=10,
        center=dict(lat=50.85, lon=4.45),
        zoom=10,
        color_continuous_scale="portland",
        mapbox_style="light",
        hover_data=["delay"],
        hover_name="stop_name",
    )
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig

# EWT per line map

In [20]:
data_g = data.groupby(
    ["route_short_name", "date_label", "trip_headsign", "delay_label_strict"],
    as_index=False
)

In [24]:
data_g

In [28]:
date_label = data[data.date_normalized == "11-09-2021"].date_label.iloc[0]

In [29]:
date_labelh

'saturday'

In [30]:
temp = data_g.get_group(("7", date_labelh, "VANDERKINDERE", "lat"))

In [31]:
temp

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
85697,113208361237049500,05:23:00,0801,18,MONTGOMERY,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,19380,NaN,cluster_0,15.0,0,2021-09-04,2021-09-11,11-09-2021,712.936477,0.0,801,6421F,VANDERKINDERE,6421,saturday,7,T,19723,05:28:43,176876,NaN,NaN,8.665992,8.540120,-0.125872,regular,343,lat,lat,lat,5,11
85700,113208050237049500,06:29:00,0801,19,MONTGOMERY,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,23340,15.0,cluster_0,15.0,0,2021-09-04,2021-09-11,11-09-2021,712.936477,0.0,801,6421F,VANDERKINDERE,6421,saturday,7,T,23424,06:30:24,176756,NaN,17.183333,8.665992,8.540120,-0.125872,regular,84,lat,lat,nan,6,11
85701,113208100237049500,06:44:00,0801,19,MONTGOMERY,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,24240,15.0,cluster_0,15.0,0,2021-09-04,2021-09-11,11-09-2021,712.936477,0.0,801,6421F,VANDERKINDERE,6421,saturday,7,T,24527,06:48:47,176805,1.0,18.383333,8.665992,8.540120,-0.125872,regular,287,lat,lat,lat,6,11
85703,113208123237049500,07:14:00,0801,19,MONTGOMERY,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,26040,15.0,cluster_0,15.0,0,2021-09-04,2021-09-11,11-09-2021,712.936477,0.0,801,6421F,VANDERKINDERE,6421,saturday,7,T,26121,07:15:21,176828,NaN,15.916667,8.665992,8.540120,-0.125872,regular,81,lat,lat,nan,7,11
85705,113208116237049500,07:44:00,0801,19,MONTGOMERY,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,27840,15.0,cluster_0,15.0,0,2021-09-04,2021-09-11,11-09-2021,712.936477,55.0,801,6421F,VANDERKINDERE,6421,saturday,7,T,28048,07:47:28,176821,NaN,19.533333,8.665992,8.540120,-0.125872,regular,208,lat,lat,lat,7,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433972,113464334238167502,22:13:00,9969F,34,CHURCHILL,6,238167502,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,79980,8.0,cluster_1,8.0,1,2021-09-18,2021-09-18,18-09-2021,73.763134,0.0,9969,6421F,VANDERKINDERE,6421,saturday,7,T,80209,22:16:49,182575,NaN,12.716667,3.898990,6.867022,2.968032,irregular,229,lat,lat,lat,22,18
3433974,113464340238167502,22:28:00,9969F,34,CHURCHILL,6,238167502,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,80880,8.0,cluster_1,8.0,1,2021-09-18,2021-09-18,18-09-2021,73.763134,0.0,9969,6421F,VANDERKINDERE,6421,saturday,7,T,80992,22:29:52,182579,NaN,10.450000,3.898990,6.867022,2.968032,irregular,112,lat,lat,nan,22,18
3433976,113464343238167502,22:43:00,9969F,34,CHURCHILL,6,238167502,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,81780,8.0,cluster_1,8.0,1,2021-09-18,2021-09-18,18-09-2021,73.763134,0.0,9969,6421F,VANDERKINDERE,6421,saturday,7,T,81895,22:44:55,182581,NaN,9.416667,3.898990,6.867022,2.968032,irregular,115,lat,lat,nan,22,18
3433978,113464351238167502,22:58:00,9969F,34,CHURCHILL,6,238167502,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,82680,8.0,cluster_1,8.0,1,2021-09-18,2021-09-18,18-09-2021,73.763134,0.0,9969,6421F,VANDERKINDERE,6421,saturday,7,T,82804,23:00:04,182586,NaN,15.150000,3.898990,6.867022,2.968032,irregular,124,lat,lat,lat,22,18


In [42]:
test = data[
    (data["route_short_name"] == "7")
    & (data.stop_name == "ROFFIAEN")
    & (data.trip_headsign == "VANDERKINDERE")
    & (data.date_normalized == "11-09-2021")
    & (data.regularity == 1)
]


In [43]:
test

,trip_id,theoretical_time,stop_id,stop_sequence,stop_name,route_id,service_id,trip_headsign,direction_id,route_short_name,route_long_name,th_time_sec,headway_th,clusters,cluster_agg_value,regularity,start_date_ft,end_date_ft,date_normalized,dist_m,distance_from_point,stop_id_cleaned,stop_id__terminus,stop_name__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,real_time_sec,adj_real_time,vehicule_id,approx_rec,headway_real,SWT,AWT,EWT,EWT_label,delay,delay_label_strict,delay_label_60,delay_label_120,hour,day
2366052,113208040237049500,09:53:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,35580,14.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,48.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,35468,09:51:08,176747,NaN,10.883333,6.096774,10.061577,3.964802,irregular,-112,adv,adv,nan,9,11
2366053,113208126237049500,10:06:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,36360,13.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,14.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,36448,10:07:28,176831,NaN,16.333333,6.096774,10.061577,3.964802,irregular,88,lat,lat,nan,10,11
2366054,113208096237049500,10:18:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,37080,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,42.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,37130,10:18:50,176801,NaN,11.366667,6.096774,10.061577,3.964802,irregular,50,nan,nan,nan,10,11
2366055,113208130237049500,10:30:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,37800,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,0.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,37835,10:30:35,176835,NaN,11.750000,6.096774,10.061577,3.964802,irregular,35,nan,nan,nan,10,11
2366056,113208127237049500,10:42:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,38520,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,0.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,38537,10:42:17,176832,1.0,11.700000,6.096774,10.061577,3.964802,irregular,17,nan,nan,nan,10,11
2366057,113208144237049500,10:54:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,39240,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,219.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,39918,11:05:18,176849,1.0,23.016667,6.096774,10.061577,3.964802,irregular,678,lat,lat,lat,10,11
2366058,113208097237049500,11:06:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,39960,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,144.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,40829,11:20:29,176802,1.0,15.183333,6.096774,10.061577,3.964802,irregular,869,lat,lat,lat,11,11
2366059,113208139237049500,11:18:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,40680,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,7.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,41551,11:32:31,176844,1.0,12.033333,6.096774,10.061577,3.964802,irregular,871,lat,lat,lat,11,11
2366060,113208131237049500,11:30:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,41400,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,157.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,41962,11:39:22,176836,1.0,6.850000,6.096774,10.061577,3.964802,irregular,562,lat,lat,lat,11,11
2366061,113208106237049500,11:42:00,5200F,25,ROFFIAEN,6,237049500,VANDERKINDERE,1,7,VANDERKINDERE - HEYSEL,42120,12.0,cluster_1,12.0,1,2021-09-04,2021-09-11,11-09-2021,576.945405,53.0,5200,6421F,VANDERKINDERE,6421,saturday,7,T,42082,11:41:22,176811,NaN,2.000000,6.096774,10.061577,3.964802,irregular,-38,adv,nan,nan,11,11


In [40]:
fig = px.scatter(
    temp,
    x="adj_real_time",
    y="headway_real",
    color="clusters",
    hover_data={'EWT':':.2f'}
).update_yaxes(
    range=(0, max(temp.headway_real)+5),
    constrain = "domain"
)

In [41]:
fig

In [32]:
fig = px.scatter(
        temp,
        x="adj_real_time",
        y="headway_real",
        color="clusters",
        hover_data={'EWT':':.2f'}
    ).update_yaxes(
        range=(0, max(temp.headway_real)+5),
        constrain = "domain"
    )


In [33]:
fig

In [9]:
theory = pd.read_pickle("data/computed/theoretical_schedule_qos.pkl")

In [20]:
temp = theory[
    (theory.route_short_name == "44")
        & (theory.stop_name == "MONTGOMERY")
        & (theory.trip_headsign == "TERVUREN STATION")
        & (theory.service_id == 233813600)
    ].sort_values(by="arrival_time")

In [21]:
fig = px.scatter(
        temp,
        x="arrival_time",
        y="headway_min",
        color="clusters",
        symbol="regularity",
        marginal_y="box",
        labels=dict(headway_min='Theoretical Headway (min)', arrival_time = 'Scheduled Time')
    )

In [22]:
fig

In [15]:
theory

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity
0,112360063233813600,05:17:00,05:17:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,19020,NaN,cluster_0,30.0,0
1,112360064233813600,05:50:00,05:50:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,21000,33.0,cluster_0,30.0,0
2,112360065233813600,06:19:00,06:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,22740,29.0,cluster_0,30.0,0
3,112360066233813600,06:49:00,06:49:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862004,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,24540,30.0,cluster_0,30.0,0
4,112360067233813600,07:19:00,07:19:00,0089,1,0,0,NaN,MONTGOMERY,NaN,50.838006,4.408970,NaN,NaN,0,37.0,40,233813600,TERVUREN STATION,0,8862002,044t0076,44,MONTGOMERY - TERVUREN STATION,NaN,0,NaN,E3BA12,000000,26340,30.0,cluster_0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819466,113543415238463502,20:43:00,20:43:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963879,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,74580,148.0,cluster_0,53.0,0
2819467,113543406238463502,21:36:00,21:36:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963876,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,77760,53.0,cluster_0,53.0,0
2819468,113543441238463502,23:35:00,23:35:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963912,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,84900,119.0,cluster_0,53.0,0
2819469,113543485238463502,23:55:00,23:55:00,9996,17,1,0,NaN,MONTGOMERY,NaN,50.837483,4.406117,NaN,NaN,0,NaN,59,238463502,MONTGOMERY,0,8963877,061b0014,61,GARE DU NORD - MONTGOMERY,NaN,3,NaN,FFDC01,000000,86100,20.0,cluster_0,53.0,0
